# RAG-based AI Teaching Assistant (Gemini)

#### Install & Import Libraries

In [ ]:
!pip install -q -U google-generativeai


In [ ]:
import google.generativeai as genai
import pandas as pd


#### Configure Gemini API Key

In [ ]:
from google.colab import userdata

api_key = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=api_key)

print("Gemini API configured successfully")


In [ ]:
def build_context(top_k_df):
    context = ""

    for _, row in top_k_df.iterrows():
        context += f"""
Lecture Title: {row['Title']}
Lecture Number: {row['Number']}
Start Time: {row['Start']} seconds
End Time: {row['End']} seconds
Lecture Content:
{row['Text']}
----------------------------------
"""
    return context


In [ ]:
def build_prompt(context, user_question):
    prompt = f"""
You are an AI teaching assistant for a Signals and Systems course.

IMPORTANT RULES:
- Use ONLY the lecture content below.
- Do NOT use outside knowledge.
- You may combine multiple lecture chunks.
- If the topic is NOT covered, say:
  "This question is not covered in the provided lectures."

You MUST:
- Explain in simple student-friendly language
- Clearly mention:
  - Lecture Title
  - Lecture Number
  - Start Time
  - End Time

Lecture Content:
{context}

User Question:
{user_question}

Answer Format (STRICT):

Answer:
<simple explanation>

Where it is taught:
- Lecture Title:
- Lecture Number:
- Start Time:
- End Time:
"""
    return prompt


In [ ]:
def gemini_rag_answer(prompt):
    model = genai.GenerativeModel("gemini-2.5-flash")
    response = model.generate_content(prompt)
    return response.text


In [ ]:
# Load the saved user question from 05_file
QUESTION_PATH = "/content/drive/MyDrive/RAG_BAS_PROJECT/user_question.json"
with open(QUESTION_PATH, "r") as f:
    user_question = json.load(f)["question"]

print(f"Loaded user question: {user_question}")

In [ ]:
import pandas as pd

TOP_K_PATH = "/content/drive/MyDrive/RAG_BAS_PROJECT/top_k.json"

# Load the top-k chunks retrieved in 05_retrieval.ipynb
top_k = pd.read_json(TOP_K_PATH)

print("Top-K retrieval data loaded successfully")
print(top_k.head())


In [ ]:

context_text = build_context(top_k)
prompt = build_prompt(context_text, user_question)

answer = gemini_rag_answer(prompt)

print("\nAI Teaching Assistant Answer:\n")
print(answer)


In [ ]:
SAVE_PATH = "/content/drive/MyDrive/RAG_BAS_PROJECT/response.txt"

with open(SAVE_PATH, "w") as f:
    f.write(answer)

print("Response saved successfully")
